In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


In [5]:
df=pd.read_csv("smartphones_dataset.csv")
df.isnull().sum()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966 entries, 0 to 965
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   brand_name           966 non-null    object 
 1   price                966 non-null    int64  
 2   rating               870 non-null    float64
 3   has_5g               966 non-null    bool   
 4   has_nfc              966 non-null    bool   
 5   has_ir_blaster       966 non-null    bool   
 6   processor_brand      946 non-null    object 
 7   num_cores            960 non-null    float64
 8   battery_capacity     955 non-null    float64
 9   ram_capacity         966 non-null    int64  
 10  internal_memory      966 non-null    int64  
 11  screen_size          966 non-null    float64
 12  num_rear_cameras     966 non-null    int64  
 13  os                   966 non-null    object 
 14  primary_camera_rear  966 non-null    float64
dtypes: bool(3), float64(5), int64(4), object

In [6]:
df['rating'].fillna(df['rating'].mode()[0],inplace=True)

df['num_cores'].fillna(df['num_cores'].mode()[0],inplace=True)
df['battery_capacity'].fillna(df['battery_capacity'].mean(),inplace=True)
df['processor_brand'].fillna(df['processor_brand'].mode()[0],inplace=True)

df.isnull().sum()

C:\Users\preet\AppData\Local\Temp\ipykernel_42940\1719802772.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['rating'].fillna(df['rating'].mode()[0],inplace=True)
C:\Users\preet\AppData\Local\Temp\ipykernel_42940\1719802772.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.


brand_name             0
price                  0
rating                 0
has_5g                 0
has_nfc                0
has_ir_blaster         0
processor_brand        0
num_cores              0
battery_capacity       0
ram_capacity           0
internal_memory        0
screen_size            0
num_rear_cameras       0
os                     0
primary_camera_rear    0
dtype: int64

In [7]:
df['Performance_Score'] = df['ram_capacity'] * (df['num_cores'] / 8)
df['Display_Quality_Score'] = df['primary_camera_rear'] / df['screen_size']
df['Battery_Efficiency_Score'] = df['battery_capacity'] / df['screen_size']
df['Memory_Ratio'] = df['ram_capacity'] / (df['internal_memory'] + 1e-6)

In [8]:
le=LabelEncoder()

bool_cols = ['has_5g', 'has_nfc', 'has_ir_blaster']



for col in bool_cols:
  if df[col].dtype=='bool':
    df[col]=le.fit_transform(df[col])

In [9]:
df

,brand_name,price,rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,battery_capacity,ram_capacity,internal_memory,screen_size,num_rear_cameras,os,primary_camera_rear,Performance_Score,Display_Quality_Score,Battery_Efficiency_Score,Memory_Ratio
0,oneplus,54999,89.0,1,1,0,snapdragon,8.0,5000.0,12,256,6.70,3,android,50.0,12.0,7.462687,746.268657,0.046875
1,oneplus,19989,81.0,1,0,0,snapdragon,8.0,5000.0,6,128,6.59,3,android,64.0,6.0,9.711684,758.725341,0.046875
2,samsung,16499,75.0,1,0,0,exynos,8.0,5000.0,4,64,6.60,3,android,50.0,4.0,7.575758,757.575758,0.062500
3,motorola,14999,81.0,1,0,0,snapdragon,8.0,5000.0,6,128,6.55,3,android,50.0,6.0,7.633588,763.358779,0.046875
4,realme,24999,82.0,1,0,0,dimensity,8.0,5000.0,6,128,6.70,3,android,108.0,6.0,16.119403,746.268657,0.046875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
961,motorola,34990,83.0,1,0,0,snapdragon,8.0,5000.0,8,128,6.67,3,android,64.0,8.0,9.595202,749.625187,0.062500
962,honor,14990,75.0,1,0,0,snapdragon,8.0,5000.0,6,128,6.50,3,android,48.0,6.0,7.384615,769.230769,0.046875
963,poco,28990,85.0,1,1,1,dimensity,8.0,5080.0,8,256,6.60,3,android,64.0,8.0,9.696970,769.696970,0.031250
964,motorola,19990,80.0,1,1,0,snapdragon,8.0,5000.0,6,128,6.80,3,android,108.0,6.0,15.882353,735.294118,0.046875


In [10]:
categorical_cols = ['brand_name', 'processor_brand', 'os']

df_encoded=pd.get_dummies(df,columns=categorical_cols ,drop_first=True,sparse=False)

In [11]:
df_encoded

,price,rating,has_5g,has_nfc,has_ir_blaster,num_cores,battery_capacity,ram_capacity,internal_memory,screen_size,...,processor_brand_helio,processor_brand_kirin,processor_brand_mediatek,processor_brand_sc9863a,processor_brand_snapdragon,processor_brand_spreadtrum,processor_brand_tiger,processor_brand_unisoc,os_ios,os_other
0,54999,89.0,1,1,0,8.0,5000.0,12,256,6.70,...,False,False,False,False,True,False,False,False,False,False
1,19989,81.0,1,0,0,8.0,5000.0,6,128,6.59,...,False,False,False,False,True,False,False,False,False,False
2,16499,75.0,1,0,0,8.0,5000.0,4,64,6.60,...,False,False,False,False,False,False,False,False,False,False
3,14999,81.0,1,0,0,8.0,5000.0,6,128,6.55,...,False,False,False,False,True,False,False,False,False,False
4,24999,82.0,1,0,0,8.0,5000.0,6,128,6.70,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
961,34990,83.0,1,0,0,8.0,5000.0,8,128,6.67,...,False,False,False,False,True,False,False,False,False,False
962,14990,75.0,1,0,0,8.0,5000.0,6,128,6.50,...,False,False,False,False,True,False,False,False,False,False
963,28990,85.0,1,1,1,8.0,5080.0,8,256,6.60,...,False,False,False,False,False,False,False,False,False,False
964,19990,80.0,1,1,0,8.0,5000.0,6,128,6.80,...,False,False,False,False,True,False,False,False,False,False


In [12]:
x=df_encoded.drop(columns="price")
y=np.log1p(df_encoded['price'])

In [13]:
# Split data into features (x) and target (y)
x = df_encoded.drop(columns="price")
y = np.log1p(df_encoded['price'])

# Get the list of feature names before splitting
feature_names = x.columns.tolist()

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(
    x, y,
    test_size=0.2,
    random_state=42 # for reproducibility
)

# Ensure the columns are in the same order for both train and test sets
x_train = x_train[feature_names]
x_test = x_test[feature_names]

In [14]:
dt_regressor = DecisionTreeRegressor(random_state=42)

param_grid = {
    'max_depth': [7, 10, 13, 16, 20, None], # Added intermediate values
    'min_samples_split': [2, 4, 8, 12],     # Expanded range
    'min_samples_leaf': [1, 2, 3, 5],       # Expanded range
    'splitter': ['best', 'random']
}

In [15]:
grid_search = GridSearchCV(
    estimator=dt_regressor,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=5,
    verbose=0, # Set to 0 to minimize output during search
    n_jobs=-1
)

grid_search.fit(x_train, y_train)

,estimator,DecisionTreeR...ndom_state=42)
,param_grid,"{'max_depth': [7, 10, ...], 'min_samples_leaf': [1, 2, ...], 'min_samples_split': [2, 4, ...], 'splitter': ['best', 'random']}"
,scoring,'neg_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'squared_error'


In [16]:
grid_search.best_params_

{'max_depth': 7,
 'min_samples_leaf': 1,
 'min_samples_split': 12,
 'splitter': 'best'}

In [17]:
grid_search.best_estimator_

,criterion,'squared_error'
,splitter,'best'
,max_depth,7
,min_samples_split,12
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,42
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,ccp_alpha,0.0


In [18]:
best_params = grid_search.best_params_
best_score = grid_search.best_score_
best_dt_model = grid_search.best_estimator_

In [19]:
# Refit the best model on the corrected training data
best_dt_model.fit(x_train, y_train)

# Make predictions on the test set
y_pred = best_dt_model.predict(x_test)

In [20]:
y_test_original = np.expm1(y_test)
y_pred_original = np.expm1(y_pred)

In [21]:
r2 = r2_score(y_test_original, y_pred_original)

In [22]:
r2

0.7087738136647868